In [7]:
import torch
from torch import nn
from torch.nn import functional as F

X = torch.rand(2, 20)

In [9]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)    # 隐藏层
        self.out = nn.Linear(256, 10)   # 输出层
    
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

net = MLP()
net(X)

tensor([[-0.0865, -0.3266, -0.0621,  0.0126, -0.1579,  0.2711, -0.1746, -0.0170,
          0.0413, -0.1582],
        [-0.0683, -0.1974, -0.0465,  0.1146, -0.1227,  0.2697, -0.0941,  0.0672,
          0.0391, -0.1280]], grad_fn=<AddmmBackward>)

In [10]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for block in args:
            self._modules[block] = block
    
    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.0149,  0.1943, -0.0874, -0.4787,  0.0006, -0.2449, -0.1600,  0.1329,
          0.0088, -0.2312],
        [-0.0819,  0.1719, -0.1735, -0.3787, -0.0147, -0.3401, -0.0255,  0.0533,
          0.0407, -0.2348]], grad_fn=<AddmmBackward>)

In [11]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)    # 该层不进行梯度的反向传播
        self.linear = nn.Linear(20, 20)
    
    def forward(self, X):
        X = self.linear(X)  # 可以实现不同层之间的参数共享
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)  # 可以实现不同层之间的参数共享
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

net = FixedHiddenMLP()
net(X)

tensor(0.0096, grad_fn=<SumBackward0>)

In [12]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(), 
                                nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)
    
    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.1581, grad_fn=<SumBackward0>)